### YOLO (You Only Look Once)

1) The YOLO framework (You Only Look Once) on the other hand, deals with object detection in a different way. 

2) It takes the entire image in a single instance and predicts the bounding box coordinates and class probabilities for these boxes. The biggest advantage of using YOLO is its superb speed – it’s incredibly fast and can process 45 frames per second. YOLO also understands generalized object representation.

### Requirements to Implement YOLO

1) <b>yolov3.weights</b><br>
Download link -> https://github.com/patrick013/Object-Detection---Yolov3/tree/master/model
    
2) <b>yolo.cfg</b><br>
Download link -> https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg
    
3)<b> coco_names</b><br>
Download link -> https://github.com/pjreddie/darknet/blob/master/data/coco.names

coco_names - It a txt file that conatins 80 different objects on which YOLO has been trained

#### Working of YOLO

1) YOLO first takes an input image.<br>
<img src="yolo1.png">
2) The framework then divides the input image into grids (say a 3 X 3 grid):<br>
<img src="yolo2.png">
3) Image classification and localization are applied on each grid. YOLO then predicts the bounding boxes and their corresponding class probabilities for objects (if any are found, of course).

#### Exmaple
Suppose we have divided the image into a grid of size 3 X 3 and there are a total of 3 classes which we want the objects to be classified into. Let’s say the classes are Pedestrian, Car, and Motorcycle respectively. So, for each grid cell, the label y will be an eight dimensional vector:

y = [pc,bx,by,bh,bw,c1,c2,c3]

where<br>
pc  - defines whether an object is present in the grid or not (it is the probability)<br>
bx, by, bh, bw  - specify the bounding box if there is an object<br>
c1, c2, c3  - represent the classes. So, if the object is a car, c2 will be 1 and c1 & c3 will be 0, and so on.

<b>Case1:</b><br>
If there is no obejcts in the grid<br>
y = [0,bx,bh,bw,bh,c1,c2,c3]<br>
Since there is no object in this grid, pc will be zero<br>

<b>Case2:</b><br>
If there is 1 or more obejcts in the grid.
Assume that the image conatins a car<br>
y = [1,bx,by,bw,bh,0,1,0]<br>
Since there is an object in this grid, pc will be equal to 1. bx, by, bh, bw will be calculated relative to the particular grid cell we are dealing with. Since car is the second class, c2 = 1 and c1 and c3 = 0. So, for each of the 9 grids, we will have an eight dimensional output vector. This output will have a shape of 3 X 3 X 8.

<img src="yolo3.png">
If we have an input image of (100,100,3) and size of output is (3,3,8)


YOLO runs both forward and backward propagation to train our model. During the testing phase, we pass an image to the model and run forward propagation until we get an output y. In real-world scenarios we take larger grids (perhaps 19 X 19) instead of 3X3.

### Calculation of bx, by, bw and bh

1)  bx, by, bh, and bw are calculated relative to the grid cell we are dealing with.
<img src="yolo4.png">
2) pc = 1 since there is an object in this grid and since it is a car, c2 = 1.
<img src="yolo5.png">
3) bx, by are the x and y coordinates of the midpoint of the object with respect to this grid. In this case, it will be (around) bx = 0.4 and by = 0.3
<img src="yolo6.png">
4) bh is the ratio of the height of the bounding box (red box in the above example) to the height of the corresponding grid cell, which in our case is around 0.9. So,  bh = 0.9. bw is the ratio of the width of the bounding box to the width of the grid cell. So, bw = 0.5 (approximately)

y = [1,0.4,0.3,0.9,0.5,0,1,0]


#### Note
bx and by will always range between 0 and 1 as the midpoint will always lie within the grid. Whereas bh and bw can be more than 1 in case the dimensions of the bounding box are more than the dimension of the grid.

### IOU and NMS

IOU - Intersection over Union<br>


NMS - Non Max Supression<br>
It suppresses significantly overlapping bounding boxes

### IOU (Intersection over Union)
1) To decide whether the predicted bounding box is giving us a good outcome (or a bad one)? This is where Intersection over Union comes into the picture. It calculates the intersection over union of the actual bounding box and the predicted buonding box.
<img src="yolo7.png">

2) The red box is the actual bounding box and the blue box is the predicted one. How can we decide whether it is a good prediction or not? IoU, or Intersection over Union, will calculate the area of the intersection over union of these two boxes. That area will be
<img src="yolo8.png">

3) IoU = Area of the intersection / Area of the union, <br>
IoU = Area of yellow box / Area of green box<br>
If IoU is greater than 0.5, we can say that the prediction is good enough. 0.5 is an arbitrary threshold we have taken here, but it can be changed according to your specific problem. Intuitively, the more you increase the threshold, the better the predictions become.

### NMS (Non-Maximum Supression)

One of the most common problems with object detection algorithms is that rather than detecting an object just once, they might detect it multiple times. In the image below, the cars are identified more than once. The Non-Max Suppression technique cleans up this up so that we get only a single detection per object.
<img src="yolo9.png">

1)  It first looks at the probabilities associated with each detection and takes the largest one. In the above image, 0.9 is the highest probability, so the box with 0.9 probability will be selected first
<img src="yolo10.png">

2) Now, it looks at all the other boxes in the image. The boxes which have low IoU with the current box are suppressed. So, the boxes with 0.6 and 0.7 probabilities will be suppressed in our example
<img src="yolo11.png">

3) After the boxes have been suppressed, it selects the next box from all the boxes with the highest probability, which is 0.8 in our case:
<img src="yolo12.png">

4) Again it will look at the IoU of this box with the remaining boxes and compress the boxes with a high IoU
<img src="yolo13.png">

5) We repeat these steps until all the boxes have either been selected or compressed and we get the final bounding boxes
<img src="yolo14.png">

This is what Non-Max Suppression is all about. We are taking the boxes with maximum probability and suppressing the close-by boxes with non-max probabilities


1) Discard all the boxes having probabilities less than or equal to a pre-defined threshold (say, 0.5)<br>
2) For the remaining boxes:<br>
a) Pick the box with the highest probability and take that as the output prediction<br>
b) Discard any other box which has IoU greater than the threshold with the output box from the above step<br>
3) Repeat step 2 until all the boxes are either taken as the output prediction or discarded


### Anchor Boxes

Each grid can only identify one object. But what if there are multiple objects in a single grid? That can so often be the case in reality. And that leads us to the concept of anchor boxes. Consider the following image, divided into a 3 X 3 grid
<img src="yolo15.png">

Remember how we assigned an object to a grid? We took the midpoint of the object and based on its location, assigned the object to the corresponding grid. In the above example, the midpoint of both the objects lies in the same grid. This is how the actual bounding boxes for the objects will be:
<img src="yolo16.png">

We will only be getting one of the two boxes, either for the car or for the person. But if we use anchor boxes, we might be able to output both boxes! How do we go about doing this? First, we pre-define two different shapes called anchor boxes or anchor box shapes. Now, for each grid, instead of having one output, we will have two outputs. We can always increase the number of anchor boxes as well. 
<img src="yolo17.png">

#### Terminologies

1)<b> Confidence: </b>Minimum probability to filter weak detections. I’ve given this a default value of 50% (0.5 )

2)<b> Threshold :</b> This is our non-maxima suppression threshold with a default value of 0.3 

3) boxes,confidences,classes = [],[],[]<br>
boxes : Our bounding boxes around the object.<br>
confidences : The confidence value that YOLO assigns to an object. Lower confidence values indicate that the object might not be what the network thinks it is. .<br>
classIDs : The detected object’s class label.<br>


In [ ]:
# 80-(0-79) indexes
# [pc,bx,by,bh,bw, 80 different categories]
# output from each grid is a vector of 1 dimension containing 85 values

#### Yolo Implementation :
https://github.com/patrick013/Object-Detection---Yolov3/blob/master/Object_Detection_Yolo.ipynb